In this notebook, CNN estimator is used as a regressor model.
This choice is leaded by the fact that :

<pre>
<li>NN model are non-linear models, then it is supposed to capture non-linear relations in between features </li>
<li>CNN model provides results while extracting semantic structures from text.</li>
</pre>

Multiple dataset are used in order to benchmark assumptions forged about model.

Data used for feeding model are read from mass storage.

Transformations that may occure over data are used only for the purpose of this notebooK.

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu(first_level=1, last_level=4, header="CNN Classifier")

# <font color=blus>0. Notebook configuration</font>

In [ ]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings('ignore')


import time
import tensorflow as tf
import pandas as pd
import gensim


import p8_util
import p8_util_config
import p9_util
import p9_util_spacy
import DataPreparator
extension='_part'

OUTPUT_DIR = './tmp/baseline'
OUTPUT_DIR_TB = './tmp'
is_tensorboard = False

#### Tensorboard processes are killed

In [ ]:
# this will kill the processes for Tensorboard
#is_tensorboard = True
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill    

    Display an item from tokenized corpus

# <font color=blus>1. Loading dataset</font>

In [ ]:
! ls -alrth ./data/DataPreparator_*.dump

## <font color=blus>1.1. Loading train DataPreparator </font>

vocab_size = len(dataPreparator_train.kerasTokenizer.tokenizer.word_index) + 1
vocab_size

In [ ]:
import numpy as np
import p5_util

filename_train = './data/DataPreparator_train_spacy_5000.dump'

if False :
    p5_util.object_dump(dataPreparator_train,filename_train)
else : 
    dataPreparator_train = p5_util.object_load(filename_train)
    
print("")
print("DataPretarator train lenth= {}".format(len(dataPreparator_train)))


vocab_size = len(dataPreparator_train.kerasTokenizer.tokenizer.word_index) + 1
print("Vocabulary size= {}".format(vocab_size))

X_train, y_train = dataPreparator_train.transform(None, None)

## <font color=blus>1.2. Loading test DataPreparator </font>

In [ ]:
#import numpy as np
import p5_util

#filename_test = './data/DataPreparator_test.dump'
filename_test = './data/DataPreparator_test_spacy_5000.dump'

if False :
    p5_util.object_dump(dataPreparator_test,filename_test)
else : 
    dataPreparator_test = p5_util.object_load(filename_test)
    
print("")
print("DataPretarator test lenth=  {}".format(len(dataPreparator_test)))


X_test, y_test = dataPreparator_test.transform(None, None)

valid_sample = X_train.shape[0]//3
valid_sample
X_test = X_test[:valid_sample,:]
y_test = y_test[:valid_sample]
print("")
print("Sample of test dataset : X_test shape= {}, y_test shape= {}".format(X_test.shape, y_test.shape))

#### Dimension reduction

In [ ]:
if False :
    X_dimension = 100
    X_train = X_train[:,:X_dimension]
    X_test  = X_test[:,:X_dimension]

In [ ]:
#print(dataPreparator_train.df_data.columns)

#### Sentences tokens counts distribution

In [ ]:
import seaborn as sns

df = dataPreparator_train.df_data['counting']
print(df.describe())
z_=sns.violinplot( y=df )

print(" ")
df = dataPreparator_test.df_data['counting']
print(dataPreparator_test.df_data['counting'].describe())
z_=sns.violinplot( y=df )

## <font color=blus>1.x. Dataset standardization </font>

In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
import p9_util
import random
if False :
    scaler, X_train_std, X_test_std = p9_util.data_scale(X_train, X_test, scaler_name='Standard')

# <font color=blus>2. Targets transformation for binary classification</font>

## <font color=blus>2.1. Conversion of targets values to binaries values</font>

`y_train` and `y_test` are vectors with float values ranging from `0.0` to `1.0`. 

`decimal_count` is the value allowing to take into account the number of decimals when converting continuous values into classes.

As a result, `y_train_label` and `y_test_label` are `[N x Classes]` matrix issued from One-hot-encoding were position of value `1` in a column indicates the class value.


In [ ]:
y_train_label_vect_bin_sample = dataPreparator_train.vectorValue2BinaryvectorLabel()
y_test_label_vect_bin_sample = dataPreparator_test.vectorValue2BinaryvectorLabel()


In [ ]:
print("\nNumber of safe texts for train sample  = {}".format(len((np.where(y_train_label_vect_bin_sample==0)[0]))))
print("Number of toxics texts for train sample= {}".format(len((np.where(y_train_label_vect_bin_sample==1)[0]))))

print("\nNumber of safe texts for test sample  = {}".format(len((np.where(y_test_label_vect_bin_sample==0)[0]))))
print("Number of toxics texts for test sample= {}".format(len((np.where(y_test_label_vect_bin_sample==1)[0]))))

In [ ]:
y_train = y_train_label_vect_bin_sample.copy()
y_test = y_test_label_vect_bin_sample.copy()
y_train.shape, y_test.shape

In [ ]:
print(y_train[0:3])
print(y_test[0:3])

## <font color=blus>1.3. Building data generator </font>

### <font color=blus>1.3.1 Making disks partitions </font>

In [ ]:
!rm ./data/train_X_*.*
!rm ./data/test_X_*.*

In [ ]:
import p9_util

partition_size = 1000
dict_train_partition, dict_train_label = p9_util.make_partition(X_train, \
                                            y_train, partition_size,\
                                            data_type="train", data_format='ndarray' )

In [ ]:
import p9_util

dict_test_partition, dict_test_label = p9_util.make_partition(X_test, \
                                              y_test,partition_size,\
                                              data_type="test", data_format='ndarray' )

### <font color=blus>1.3.2 Building train and test `DataGenerator` </font>

In [ ]:
import p9_util

embedding_dim=0
dimension = X_train.shape[1]
params = {'dim': (dimension,embedding_dim),
          'batch_size': 100,
          'n_classes': 2,
          'n_channels': 0,
          'shuffle': False}
print(params)

# Data Generators
import DataGenerator
len_train = X_train.shape[0]
train_generator = DataGenerator.DataGenerator(dict_train_partition, dict_train_label, partition_size, len_train,**params)

len_test = X_test.shape[0]
test_generator = DataGenerator.DataGenerator(dict_test_partition, dict_test_label, partition_size,len_test, **params)

In [ ]:
!rm ./data/*_generator_*.dump

In [ ]:
import p5_util
nb_class = params['n_classes']

nb_row = X_train.shape[0]
filename_train = "./data/train_generator_"+str(nb_row)+"_"+str(nb_class)+".dump"

nb_row = X_test.shape[0]
filename_test = "./data/test_generator_"+str(nb_row)+"_"+str(nb_class)+".dump"

p5_util.object_dump(train_generator, filename_train)
p5_util.object_dump(test_generator, filename_test)

# <font color=blus>3. CNN processing using Keras</font>

    Convolutional layers are built with strides >0 and BOW representation of texts.
    This is equivalent to process Bag Of Words with n-gram.


## <font color=blus>3.1. Configuration of classifier model</font>

    Print some parameters for CNN learning

In [ ]:
X_train.shape

In [ ]:
print("\nX_train_bow shape = {}".format(X_train.shape))
print("Y train shape =       {}".format(y_train.shape))
print()

print("X_test_bow shape  = {}".format(X_test.shape))
print("Y test shape =      {}".format(y_test.shape))
print()
print("Vocabulary size= {}".format(vocab_size))

In [ ]:
print(" ")
print("Batch size= {}".format(params['batch_size']))   
vector_dimension = params['dim'][0]
print("Input data size= {}".format(vector_dimension))    

## <font color=blus>2.2 Modeling a regression model</font>

### <font color=blus>2.2.1. Loading train and test data generators</font>

In [ ]:
!ls -alrth ./data/*_generator_*.dump

## <font color=blus>2.2.2. CNN classifier building</font>

In [ ]:
import p5_util

filename_train = "./data/train_generator_4571_2.dump"

filename_test = "./data/test_generator_1523_2.dump"

train_generator = p5_util.object_load( filename_train)
test_generator = p5_util.object_load( filename_test)

In [ ]:
#dir(train_generator)
dict_param = train_generator.get_params()


In [ ]:
dict_param= {'dim': (300, 0),
 'batch_size': 100,
 'n_classes': 2,
 'n_channels': 0,
 'shuffle': False}

In [ ]:

vector_dimension = dict_param['dim'][0]
nbClasses = dict_param['n_classes']
nb_channel = dict_param['n_channels']

dict_param

Ouptput layer dimensions for convolutional network is computed as following : 

*  <code>C = [(I-F + 2*P)/S] +1</code> where :
    * <code>C</code> is the dimension of the ouput convolutional layer;
    * <code>I</code> is the input size
    * <code>F</code> is the convolutional filter size
    * <code>P</code> is the padding size (0 here)
    * <code>S</code> is the srtides

In [ ]:
filter_size = 3
strides = filter_size-1

print("Expected defaut dimension for 1st hidden layer : {}".format(round((vector_dimension-filter_size+2*0)/strides +1)))

Strides value of <code>S</code> allows to regard a bulk of <code>S</code> consecutive words for sentences patterns exploration. 

For compilation, a learning rate of <code>0.01</code> increases the speed of learning. 

Batch normalization insures a smooth learning.

Batch size is selected in a such way an epoch is formed with 10 mini-batches.

In [ ]:
import p9_util
nb_channel=0
model = p9_util.keras_cnn_build(dict_param['dim'],nbClasses=nbClasses, \
                                strides=strides, nb_filter=128, filter_size=strides, \
                                lr=1.e-2, dropout_rate=0., \
                                conv_layer=1,nb_dense_neuron=128,\
                                dense_layer=2)
model.summary()

In [ ]:
# Train model on dataset
history = model.fit_generator(generator=train_generator,
                    validation_data=test_generator,
                    use_multiprocessing=False,
                    workers=1, verbose=1, epochs=10)

#### CNN model and history are either saved or restored depending a Boolean flag.

`core_name` is a parameter identifying a model through its name.

This name may be changed depending model parameters.

In [ ]:
core_name = "cnn_class_6layers"

In [ ]:
import keras
import p5_util

if True:
    model.save("./data/model_"+core_name+".h5")
    p5_util.object_dump(history, "history_"+core_name+".dump")
    #p5_util.object_dump(max_length, "max_length_"+core_name+".dump")

In [ ]:
import keras
import p5_util

if False :
    pass
else :
    model = keras.models.load_model("./data/model_"+core_name+".h5")
    history = p5_util.object_load("history_"+core_name+".dump")
    #max_length = p5_util.object_load("max_length_"+core_name+".dump")

#print("\nInput data length= {}".format(max_length))
model.summary()

In [ ]:
history.history

    Metric and loss curves are displayed for train and test dataset.

In [ ]:
for key, values in history.history.items() :
    history.history[key] = values[1:]
history.history

In [ ]:
import p7_util
max_length=''
list_model_acc_value = p7_util.p7_plot_cnn_history(model, None, None, history=history, legend='length='+str(max_length))

In [ ]:
import p7_util
max_length=''
list_model_acc_value = p7_util.p7_plot_cnn_history(model, None, None, history=history, legend='length='+str(max_length))

## <font color=blus>2.3. Words embeddings</font>

## <font color=blus>2.3.1. Building embeddings with Glove</font>

Dictionary `{word:glove_coefficient}` is built from Glove file name.

Glove file has been prealably downloaded.

Once built, dictionary allows to build a vector for every word 
in vocabulary issued from tokenizer.

Using glove file defined here-under, each word is a vector of dimension 100. This dimension is 
referenced in the Glove file name part as `100d`

Endly, weights matrix is built from vocabulary issued from tokenizer. 

Such process is summarized with sequences here-under :
    * dict_glove_word_coeff <-- processing Glove file name
    * vocabulary_word, index <-- tokenizer
    * weight_vector = dict_glove_word_coeff[vocabulary_word]
    * weight_matrix[index] = weight_vector

In [ ]:
import numpy as np
import p5_util

filename = './data/embbeding_glove.dump'

weight_matrix = p5_util.object_load(filename)
    
print("")
print("Embedding shape=  {}".format(weight_matrix.shape))


In [ ]:
import p9_util

embedding_dim=100
dimension = X_train.shape[1]
params = {'dim': (dimension,embedding_dim),
          'batch_size': 100,
          'n_classes': 0,
          'n_channels': 0,
          'shuffle': False}
print(params)

In [ ]:
# Data Generators
import DataGenerator
len_train = X_train.shape[0]
train_generator = DataGenerator.DataGenerator(dict_train_partition, dict_train_label, partition_size, len_train,**params)

len_test = X_test.shape[0]
test_generator = DataGenerator.DataGenerator(dict_test_partition, dict_test_label, partition_size,len_test, **params)

In [ ]:
filter_size = 9
strides = filter_size-1
nb_filter=128
print("Expected defaut dimension for 1st hidden layer : {}".format(round((vector_dimension-filter_size+2*0)/strides +1)))

In [ ]:
vector_dimension
vocab_size=weight_matrix.shape[0]


In [ ]:
import p9_util
embedding_dim=100
model = p9_util.keras_cnn_build((vector_dimension, embedding_dim),nbClasses=1, \
                                strides=strides, nb_filter=nb_filter, filter_size=strides, lr=1.e-2,\
                               isWordEmbedding = True, vocab_size = vocab_size,weight_matrix=weight_matrix)
model.summary()

In [ ]:
# Train model on dataset
history = model.fit_generator(generator=train_generator,
                    validation_data=test_generator,
                    use_multiprocessing=False,
                    workers=4, verbose=1, epochs=20)

## <font color=blus>2.3.2. Building embeddings with Scapy</font>

In [ ]:
def scapy_weight_matrix_build(spacy_nlp,llist_token_string):
    set_token_string = set()
    index = 0
    
    for list_token_string in llist_token_string :
        for token_string in list_token_string :
            set_token_string.add(token_string)

    vocab_size = len(set_token_string)
    vector_dimension = spacy_nlp.vocab.vectors_length
    weight_matrix = np.zeros((vocab_size, vector_dimension))
    dict_index_tokenstring = dict()
    for token_string, index in zip(set_token_string, range(vocab_size)) :
        if nlp.vocab.has_vector(token_string):
            weight_matrix[index] = nlp.vocab.get_vector(token_string)
            dict_index_tokenstring[index] = token_string
    return weight_matrix,dict_index_tokenstring

In [ ]:
import spacy
import p9_util


nlp = p9_util.SPACY_NLP_MD
print(nlp.vocab.length)


In [ ]:
column_name = dataPreparator_train.COLUMN_NAME_TOKEN
llist_token = dataPreparator_train.df_data[column_name].tolist()
spacy_weigth_matrix, dict_index_token = scapy_weight_matrix_build(nlp,llist_token_string)

In [ ]:
import p9_util

embedding_dim=300
dimension = spacy_weigth_matrix.shape[1]
params = {'dim': (dimension,embedding_dim),
          'batch_size': 100,
          'n_classes': 0,
          'n_channels': 0,
          'shuffle': False}
print(params)

In [ ]:
# Data Generators
import DataGenerator
len_train = X_train.shape[0]
train_generator = DataGenerator.DataGenerator(dict_train_partition, dict_train_label, partition_size, len_train,**params)

len_test = X_test.shape[0]
test_generator = DataGenerator.DataGenerator(dict_test_partition, dict_test_label, partition_size,len_test, **params)

In [ ]:
filter_size = 9
strides = filter_size-1
nb_filter=128
print("Expected defaut dimension for 1st hidden layer : {}".format(round((vector_dimension-filter_size+2*0)/strides +1)))

In [ ]:
vector_dimension
vocab_size=weight_matrix.shape[0]
vector_dimension,vocab_size,embedding_dim

In [ ]:
import p9_util
embedding_dim=100
model = p9_util.keras_cnn_build((vector_dimension, embedding_dim),nbClasses=1, \
                                strides=strides, nb_filter=nb_filter, filter_size=strides, lr=1.e-2,\
                               isWordEmbedding = True, vocab_size = vocab_size,weight_matrix=weight_matrix)
model.summary()

In [ ]:
# Train model on dataset
history = model.fit_generator(generator=train_generator,
                    validation_data=test_generator,
                    use_multiprocessing=False,
                    workers=4, verbose=1, epochs=20)